<a href="https://colab.research.google.com/github/imabari/musen/blob/gh-pages/rakuten5Gnara.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import csv
import datetime
import pathlib
import re
import time
import urllib.parse

import pandas as pd
import requests

In [2]:
api = {
    # 1:免許情報検索  2: 登録情報検索
    "ST": 1,
    # 詳細情報付加 0:なし 1:あり
    "DA": 1,
    # スタートカウント
    "SC": 1,
    # 取得件数
    "DC": 2,
    # 出力形式 1:CSV 2:JSON 3:XML
    "OF": 1,
    # 無線局の種別
    "OW": "FB",
    # 所轄総合通信局
    "IT": "E",
    # 都道府県
    "HCV": 29000,
    # 周波数（始）
    "FF": 3849.99,
    # 周波数（単位）
    "HZ": 2,
    # 免許人名称/登録人名称
    "NA": "楽天モバイル",
}

In [3]:
sc = 1
n = 0

dfs = []

while True:

    api["SC"] = sc
    parm = urllib.parse.urlencode(api, encoding="shift-jis")

    r = requests.get("https://www.tele.soumu.go.jp/musen/list", parm)
    r.raise_for_status()

    cr = csv.reader(r.text.splitlines(), delimiter=",")
    data = list(cr)

    n = int(data[0][1])

    df_tmp = pd.DataFrame(data[1:]).dropna(how="all")
    dfs.append(df_tmp)

    sc += 500

    print(sc, n)

    if sc > n:
        break
 
    time.sleep(1)

501 12


In [4]:
df = pd.concat(dfs).reset_index(drop=True)

In [5]:
df[0] = df.index

In [6]:
df[23] = df[23].str.replace(r"\\n", "\n").apply(lambda s: [i.strip() for i in re.split("(?<=W\n)", s)])

In [7]:
df1 = df.explode(23).reset_index(drop=True)

In [8]:
df1

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25
0,0,楽天モバイル株式会社 法人番号2010901041404,奈良県奈良市,電気通信業務用,2020-12-07,楽天モバイル株式会社,＊＊＊＊＊,,基地局,＊＊＊＊＊,2020-12-07,2022-09-30まで,電気通信業務用,,常 時,,,,電気通信業務に関する事項,免許人所属の陸上移動局,*****,送受信所\n奈良県奈良市\n\n制御所\n神奈川県横浜市戸塚区\n\n監視所\n東京都港区,,99M9X7W\t3849.99 MHz\t654 mW,,
1,1,楽天モバイル株式会社 法人番号2010901041404,奈良県奈良市,電気通信業務用,2020-12-07,楽天モバイル株式会社,＊＊＊＊＊,,基地局,＊＊＊＊＊,2020-12-07,2022-09-30まで,電気通信業務用,,常 時,,,,電気通信業務に関する事項,免許人所属の陸上移動局,*****,送受信所\n奈良県奈良市\n\n制御所\n神奈川県横浜市戸塚区\n\n監視所\n東京都港区,,100MX7W\t27.05004 GHz\t663 mW,,
2,1,楽天モバイル株式会社 法人番号2010901041404,奈良県奈良市,電気通信業務用,2020-12-07,楽天モバイル株式会社,＊＊＊＊＊,,基地局,＊＊＊＊＊,2020-12-07,2022-09-30まで,電気通信業務用,,常 時,,,,電気通信業務に関する事項,免許人所属の陸上移動局,*****,送受信所\n奈良県奈良市\n\n制御所\n神奈川県横浜市戸塚区\n\n監視所\n東京都港区,,100MX7W\t27.15 GHz\t663 mW,,
3,1,楽天モバイル株式会社 法人番号2010901041404,奈良県奈良市,電気通信業務用,2020-12-07,楽天モバイル株式会社,＊＊＊＊＊,,基地局,＊＊＊＊＊,2020-12-07,2022-09-30まで,電気通信業務用,,常 時,,,,電気通信業務に関する事項,免許人所属の陸上移動局,*****,送受信所\n奈良県奈良市\n\n制御所\n神奈川県横浜市戸塚区\n\n監視所\n東京都港区,,100MX7W\t27.24996 GHz\t663 mW,,
4,1,楽天モバイル株式会社 法人番号2010901041404,奈良県奈良市,電気通信業務用,2020-12-07,楽天モバイル株式会社,＊＊＊＊＊,,基地局,＊＊＊＊＊,2020-12-07,2022-09-30まで,電気通信業務用,,常 時,,,,電気通信業務に関する事項,免許人所属の陸上移動局,*****,送受信所\n奈良県奈良市\n\n制御所\n神奈川県横浜市戸塚区\n\n監視所\n東京都港区,,100MX7W\t27.34992 GHz\t663 mW,,
5,2,楽天モバイル株式会社 法人番号2010901041404,奈良県奈良市,電気通信業務用,2021-02-25,楽天モバイル株式会社,＊＊＊＊＊,,基地局,＊＊＊＊＊,2021-02-25,2022-09-30まで,電気通信業務用,,常 時,,,,電気通信業務に関する事項,免許人所属の陸上移動局,*****,送受信所\n奈良県奈良市\n\n制御所\n神奈川県横浜市戸塚区\n\n監視所\n東京都港区,,100MX7W\t27.05004 GHz\t663 mW,,
6,2,楽天モバイル株式会社 法人番号2010901041404,奈良県奈良市,電気通信業務用,2021-02-25,楽天モバイル株式会社,＊＊＊＊＊,,基地局,＊＊＊＊＊,2021-02-25,2022-09-30まで,電気通信業務用,,常 時,,,,電気通信業務に関する事項,免許人所属の陸上移動局,*****,送受信所\n奈良県奈良市\n\n制御所\n神奈川県横浜市戸塚区\n\n監視所\n東京都港区,,100MX7W\t27.15 GHz\t663 mW,,
7,2,楽天モバイル株式会社 法人番号2010901041404,奈良県奈良市,電気通信業務用,2021-02-25,楽天モバイル株式会社,＊＊＊＊＊,,基地局,＊＊＊＊＊,2021-02-25,2022-09-30まで,電気通信業務用,,常 時,,,,電気通信業務に関する事項,免許人所属の陸上移動局,*****,送受信所\n奈良県奈良市\n\n制御所\n神奈川県横浜市戸塚区\n\n監視所\n東京都港区,,100MX7W\t27.24996 GHz\t663 mW,,
8,2,楽天モバイル株式会社 法人番号2010901041404,奈良県奈良市,電気通信業務用,2021-02-25,楽天モバイル株式会社,＊＊＊＊＊,,基地局,＊＊＊＊＊,2021-02-25,2022-09-30まで,電気通信業務用,,常 時,,,,電気通信業務に関する事項,免許人所属の陸上移動局,*****,送受信所\n奈良県奈良市\n\n制御所\n神奈川県横浜市戸塚区\n\n監視所\n東京都港区,,100MX7W\t27.34992 GHz\t663 mW,,
9,3,楽天モバイル株式会社 法人番号2010901041404,奈良県奈良市,電気通信業務用,2021-02-25,楽天モバイル株式会社,＊＊＊＊＊,,基地局,＊＊＊＊＊,2021-02-25,2022-09-30まで,電気通信業務用,,常 時,,,,電気通信業務に関する事項,免許人所属の陸上移動局,*****,送受信所\n奈良県奈良市\n\n制御所\n神奈川県横浜市戸塚区\n\n監視所\n東京都港区,,100MX7W\t27.05004 GHz\t663 mW,,


In [9]:
def bunri(s):

    d = re.split(r"\\t", s)

    return [d[0], " ".join(d[1:-1]), d[-1]]

In [10]:
df2 = df1[23].apply(bunri).apply(pd.Series).rename(columns={0: "電波の型式", 1: "周波数", 2: "空中線電力"})

In [11]:
df2

,電波の型式,周波数,空中線電力
0,99M9X7W,3849.99 MHz,654 mW
1,100MX7W,27.05004 GHz,663 mW
2,100MX7W,27.15 GHz,663 mW
3,100MX7W,27.24996 GHz,663 mW
4,100MX7W,27.34992 GHz,663 mW
5,100MX7W,27.05004 GHz,663 mW
6,100MX7W,27.15 GHz,663 mW
7,100MX7W,27.24996 GHz,663 mW
8,100MX7W,27.34992 GHz,663 mW
9,100MX7W,27.05004 GHz,663 mW


In [12]:
df2["周波数"] = df2["周波数"].apply(lambda s: " ".join(s.split()))

In [13]:
df2["空中線電力"] = df2["空中線電力"].apply(lambda s: " ".join(s.split()))

In [14]:
df3 = df1.iloc[:, [0, 2]].join(df2).rename(columns={0: "グループ", 2: "市区町村"})
df3

,グループ,市区町村,電波の型式,周波数,空中線電力
0,0,奈良県奈良市,99M9X7W,3849.99 MHz,654 mW
1,1,奈良県奈良市,100MX7W,27.05004 GHz,663 mW
2,1,奈良県奈良市,100MX7W,27.15 GHz,663 mW
3,1,奈良県奈良市,100MX7W,27.24996 GHz,663 mW
4,1,奈良県奈良市,100MX7W,27.34992 GHz,663 mW
5,2,奈良県奈良市,100MX7W,27.05004 GHz,663 mW
6,2,奈良県奈良市,100MX7W,27.15 GHz,663 mW
7,2,奈良県奈良市,100MX7W,27.24996 GHz,663 mW
8,2,奈良県奈良市,100MX7W,27.34992 GHz,663 mW
9,3,奈良県奈良市,100MX7W,27.05004 GHz,663 mW


In [15]:
df4 = pd.crosstab([df3["グループ"], df3["市区町村"]], df3["周波数"])
df4

,周波数,27.05004 GHz,27.15 GHz,27.24996 GHz,27.34992 GHz,3849.99 MHz
グループ,市区町村,,,,,
0,奈良県奈良市,0,0,0,0,1
1,奈良県奈良市,1,1,1,1,0
2,奈良県奈良市,1,1,1,1,0
3,奈良県奈良市,1,1,1,1,0
4,奈良県奈良市,0,0,0,0,1
5,奈良県奈良市,1,1,1,1,0
6,奈良県奈良市,0,0,0,0,1
7,奈良県奈良市,0,0,0,0,1
8,奈良県橿原市,1,1,1,1,0


In [16]:
df4.to_csv("rakuten5G.csv", encoding="utf_8_sig")

In [17]:
from google.colab import files

files.download("rakuten5G.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>